# Coding System

> Aim: create a agent that takes the plan and writes code to solve the plan. The coding agent will have to follow a specific format for the code so that the code can be exectured and run by the debugging agent.

In [1]:
# Puzzle: 2024 day 4
PUZZLE = """
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:


..X...
.SAMX.
.A..A.
XMAS.S
.X....
The actual word search will be full of letters instead. For example:

MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:

....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
Take a look at the little Elf's word search. How many times does XMAS appear?
"""

## Setup the notebook

In [7]:
import os
import sys
from dotenv import load_dotenv

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.path.abspath(""), 'src/')


print(PROJECT_ROOT)

sys.path.append(PROJECT_ROOT)

from models.gemini_model import GeminiLanguageModel
from agents.pre_processing_agent import PreProcessingAgent
from agents.retreival_agent import RetrievalAgent
from agents.planning_agent import PlanningAgent 
from agents.base_agent import MockAgent, BaseAgent
from utils.util_types import AgentSettings, Puzzle
from core.orchestrator import Orchestrator
from core.state import MainState

/home/twanh/workspace/thesis/thesis-advent-of-agents/src/


In [3]:
load_dotenv()

True

In [4]:
# Create a model that can be used
model = GeminiLanguageModel(
    api_key=os.getenv("GEMINI_API_KEY"),
    model_name='gemini-2.0-flash'
)

In [8]:
agents: tuple[tuple[BaseAgent, AgentSettings], ...] = (
    (
        PreProcessingAgent('preprocess', model=model),
        AgentSettings(enabled=True, can_debug=False),
    ),
    (
        RetrievalAgent(
            'retreival',
            model=model,
            connection_string=os.getenv('DB_CONNECTION_STRING') or '',
            openai_key=os.getenv('OPENAI_API_KEY') or '',
        ),
        AgentSettings(enabled=True, can_debug=False),
    ),
    (
        PlanningAgent('planning', model=model),
        AgentSettings(enabled=True, can_debug=False),
    )
)


orchestrator = Orchestrator(agents, {})


puzzle = Puzzle(
    description=PUZZLE,
    solution=None,
    year=2024,
    day=5,
)

state = MainState(puzzle=puzzle)

return_state = orchestrator.solve_puzzle(state)

2025-05-07 13:49:14.410 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-07 13:49:14.411 | INFO     | core.orchestrator:solve_puzzle:56 - Running agent: preprocess
2025-05-07 13:49:14.412 | DEBUG    | agents.pre_processing_agent:process:21 - Preprocessing agent prompt: You are a pre-processing agent. Your task is to process the following Advent of Code puzzle description so that the output can be stored in a RAG (Retrieval-Augmented Generation) database and later used by a planning agent. Follow these steps precisely:

------------------------------------------------------------
Step 1: Detailed Extraction
------------------------------------------------------------
Extract every technical detail from the puzzle and separate it from the narrative. Specifically, please extract:
  - Core Problem Statement:
      - A concise description of the computational task (exclude story or decorative language).
  - Input Specifications:
     - Detailed description

In [9]:
from pprint import pprint, pformat

pprint(return_state)

MainState(puzzle=Puzzle(description='\n"Looks like the Chief\'s not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!\n\nAs the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she\'d like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.\n\nThis word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It\'s a little unusual, though, as you don\'t merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:\n\n\n..X...\n.SAMX.\n.A..A.\nXMAS.S\n.X....\nThe actual word search will be full of letters instead. For example:\n\nMMMSXXMASM\nMSAMXMSMSA\nAMXSXMAAMM\nMSAMASMSMX\nXMASAMXAMM\nXXAMMXXAMA\nSMSMSASXSS\nSAXAMA

# Coding Agent

In [ ]:
PROMPT = """
# Advent of Code Implementation Agent

You are an expert coding agent specializing in implementing solutions for Advent of Code puzzles. 
Your task is to convert a detailed solution plan into clean, efficient, and correct Python code that solves the given problem. 
You excel at translating algorithmic plans into precise implementations.

## INPUT FORMAT

You will receive:

1. A problem statement from Advent of Code
2. A detailed solution plan generated by a planning agent
3. Extra information such as input output format, test inputs and expected test outputs
4. Any example inputs/outputs from the original problem
5. Similar (older) problems with example solution code (which may be in a different programming language)

It will be provided as the following JSON

```json
{{
    "problem_statement": "The problems statement",
    "full_description": "The full description of the problem (string)",
    "underlying_concepts": ["string"],
    "keywords": ["string"],
    "input_format": "The input format (string)",
    "output_format": "The expected output (string)",
    "constraints": ["string"],
    "example_solutions": [
            {{
            "puzzle": "Previous puzzle title or brief description",
            "code": "The code used to solve this previous puzzle (can be empty if not available)"
            }}
    ],
    "plan": "The plan you should follow to solve the problem (string)",
    "test_cases": "Simple test cases that can you help reason if your code is correct (string)"
}}
```

## YOUR RESPONSIBILITIES

Your primary goal is to produce a complete, correct, and efficient Python implementation that:

1. Correctly solves both the provided examples and will work for the actual puzzle input
2. Follows good software engineering practices
3. Includes appropriate comments and documentation
4. Handles edge cases and potential errors
5. Is executable via command line as: `python3 [program].py [puzzleinputfile]`

## IMPLEMENTATION PROCESS

Follow these steps meticulously:

-----------------------------------------
STEP 1. Analyze the Plan
-----------------------------------------

- Thoroughly read and understand the provided solution plan (key: `plan` in your input)
- Identify any ambiguities, missing details, or potential issues in the plan
- Mentally trace through the examples to ensure you understand the expected flow


-----------------------------------------
STEP 2. Design Your Code Structure
-----------------------------------------

- Create a clear, modular structure with well-named functions matching the plan's major steps
- Define appropriate data structures with explicit type hints
- Plan your function signatures and interfaces before implementation
- Use the keywords and underlying concepts to think about what algoritms to use to solve the problems.

-----------------------------------------
STEP 3. Implement Core Logic
-----------------------------------------

- Write robust implementations of all algorithms described in the plan
- Include detailed comments explaining complex logic
- Follow Python best practices (PEP 8, appropriate naming conventions)
- Use type hints throughout your code


-----------------------------------------
STEP 4. Handle Edge Cases Explicitly
-----------------------------------------

- Add specific code to handle all edge cases mentioned in the plan
- Anticipate and handle additional edge cases common in Advent of Code:
  - Empty input
  - Boundary conditions (min/max values)
- Use the test cases to reason about your code and make sure it would solve the test cases correctly

----------------------------------------
STEP 5. Test Against Examples
----------------------------------------

- Include code that runs and validates against all provided examples
- Add assertions to verify intermediate results match expected values
- Print debugging information that would help diagnose issues to STDERR
    - STDOUT can only be used to print the final result.

----------------------------------------
STEP 6. Optimize If Necessary
-----------------------------------------

- Review your solution for performance bottlenecks
- Apply optimizations where appropriate, explaining your choices
- Ensure the solution will scale to handle the full problem input

-----------------------------------------
STEP 7. Finalize Solution
-----------------------------------------

- Ensure your code has a clear entry point (typically a `main()` function)
- Include code to read from the puzzle input file specified as a command-line argument
- Make sure that your code follows the proper structure as documented (example code template) below.
- Add a brief summary comment at the top explaining the approach
- Verify all functions have appropriate docstrings


-----------------------------------------
OUTPUT FORMAT
-----------------------------------------
Your response must be a valid JSON object with the following structure:

```json
{
  "code": "Complete Python code as a string with all necessary formatting"
}
```

Ensure your code in the "code" field:
1. Is properly escaped for JSON inclusion
2. Maintains proper indentation and formatting
3. Is immediately executable as `python3 [program].py [puzzleinputfile]`

## REFERENCE SOLUTION (IF PROVIDED)

If a reference solution to a similar problem has been provided, study it carefully to understand:
- How the code is structured into functions
- How input parsing is handled
- How the core algorithms are implemented
- How edge cases are managed
- How the solution is tested

## EXAMPLE CODE TEMPLATE

```python
\"\"\"
Advent of Code [Year] Day [Number]: [Title]
Solution implementation based on the provided plan.

Usage: python3 solution.py [input_file]
\"\"\"
from typing import List, Dict, Tuple, Set, Optional
import sys
from collections import defaultdict, deque
import re
# Import other necessary libraries

def parse_input(input_file: str) -> [appropriate_return_type]:
    "\"\"Parse the puzzle input from file into appropriate data structures.
    
    Args:
        input_file: Path to the input file
        
    Returns:
        [Description of return value]
    \"\"\"
    with open(input_file, 'r') as f:
        # Process file content
        pass
    # Implementation...
    
def solve_part_one(parsed_data: [type]) -> [type]:
    \"\"\"Solve part one of the puzzle.
    
    Args:
        parsed_data: Processed input data
        
    Returns:
        Solution for part one
    \"\"\"
    # Implementation...
    
def main():
    # Check command line arguments
    if len(sys.argv) < 2:
        print("Usage: python3 solution.py [input_file]")
        return
    
    input_file = sys.argv[1]
    
    # Parse input
    parsed_data = parse_input(input_file)
    
    # Solve part one
    part_one_solution = solve_part_one(parsed_data)
    print(f"Part One: {{part_one_solution}}")
    
    # Test with examples (if available)
    # [Example testing code]

if __name__ == "__main__":
    main()
```

Remember to follow the plan closely while filling in implementation details that the planner may have omitted. Your goal is to bridge the gap between algorithmic description and working code.


-----------------------------------

Your input is:

{json_input}
"""

### Notes

Should I give all this information to the coding agent? Or only plan/input/output test cases.

In [14]:
# Construct the json_input
import json

# "problem_statement": "The problems statement",
# "full_description": "The full description of the problem (string)",
# "underlying_concepts": ["string"],
# "keywords": ["string"],
# "input_format": "The input format (string)",
# "output_format": "The expected output (string)",
# "constraints": ["string"],
# "example_solutions": [
#         {{
#         "plan": "Previous puzzle plan or description",
#         "code": "The code used to solve this previous puzzle (can be empty if not available)"
#         }}
# ],
# "plan": "The plan you should follow to solve the problem (string)",
# "test_cases": "Simple test cases that can you help reason if your code is correct (string)"

inp = {
    "problem_statement": return_state.problem_statement,
    "full_description": return_state.puzzle.description,
    "underlying_concepts": return_state.underlying_concepts,
    "keywords": return_state.keywords,
    "input_format": return_state.input_format,
    "output_format": return_state.output_format,
    "constraints": return_state.constraints,
    "example_solutions": [{"plan": i[1], "code": i[0].solution} for i in return_state.retreived_puzzles],
    "plan": return_state.selected_plan.plan,
    "test_cases": return_state.test_cases
}

json_input = json.dumps(inp, indent=2)
print(json_input)

{
  "problem_statement": "Count the number of times the word 'XMAS' appears in a grid of characters. The word can be oriented horizontally, vertically, diagonally, or backwards, and can overlap with itself or other instances of the word.",
  "full_description": "\n\"Looks like the Chief's not here. Next!\" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!\n\nAs the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.\n\nThis word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been 